<h1 style = "font-size: 30px; text-align: center;">AI Genetic Hands On</h1>
<h2 style = "font-size: 25px; text-align: center;">Hospital Job Scheduling</h2>
<h2 style = "font-size: 25px; text-align: center; color: #666">Name: Alireza Aghaei </h2>
<h2 style = "font-size: 25px; text-align: center; color: #666">Student Id: 810197679 </h2>
<h4 style="text-align: center">Spring 1400</h4>

<h2 style = "font-size: 25px;">Notes</h2>

<h3>Tests: </h3>

<p style="text-indent :2em;"><b>Test1: </b><mark>Average time < 15s</mark></p>
<p style="text-indent :2em;"><b>Test2: </b><mark>Average time < 60s</mark></p>
    

<h3>Outputs: </h3>
<p style="text-indent :2em;">Respectively <mark>output1.txt</mark> and <mark>output2.txt</mark></p>

<br>

<h2 style = "font-size: 25px;">Test Files</h2>

In [1]:
import random
testFile1 = "test1.txt"
testFile2 = "test2.txt"
outputFile1 = "output1.txt"
outputFile2 = "output2.txt"

<h2 style = "font-size: 25px;">Reading Test File Content</h2>

In [2]:
def readInput(testFile) :
    file = open(testFile, 'r+')
    fileList = file.readlines()
    fileList = [s.replace('\n', '') for s in fileList]
    
    [days, doctors] = [int(i) for i in fileList[0].split()]
    maxCapacity = int(fileList[1])
    
    allShifts = []
    for i in range(2, days + 2):
        dayRequirements = fileList[i].split()
        morningReqs = [int(i) for i in dayRequirements[0].split(",")]
        eveningReqs = [int(i) for i in dayRequirements[1].split(",")]
        nightReqs = [int(i) for i in dayRequirements[2].split(",")]
        
        allShifts.append((morningReqs, eveningReqs, nightReqs))

    file.close()
    return [days, list(range(doctors)), maxCapacity, allShifts]

<h2 style = "font-size: 25px;">Job Scheduler</h2>

In [3]:
class JobScheduler:
    def __init__(self, fileInfo):
        self.days = fileInfo[0]
        self.doctors = len(fileInfo[1])
        self.doctorsIds = fileInfo[1]
        self.maxCapacity = fileInfo[2]
        self.allShifts = fileInfo[3]

        # You can use these values for your genetic algorithm
        self.popSize = 300
        self.crossOverPoints = self.days#(relative to number of days)
        self.elitismPercentage = 3 #(move x% best of parents directly to the new population)
        self.pc = 0.6 #(crossover probability) 0.65
        self.pm = 0.004  #(mutation probability) 0.4
        self.goalFitnessVal = 3 * self.doctors + self.days
        self.chromosomes = self.generateInitialPopulation()
        
        
    def generateInitialPopulation(self):
        result = []
        for i in range(self.popSize):
            new_pop = []
            for j in range(3 * self.days * self.doctors):
                new_pop.append(random.randint(0 , 1))
            result.append(new_pop)
        return result
        
    def onePointCrossOver(self, gene1 , gene2):
        crossOverPoint = random.randint(0 , 3 * self.days - 2)
        backup = gene1[crossOverPoint + 1 : len(gene1)]
        gene1[crossOverPoint + 1 : len(gene1)] = gene2[crossOverPoint + 1 : len(gene2)].copy()
        gene2[crossOverPoint + 1 : len(gene1)] = backup.copy()
        return gene1, gene2

    def crossOver(self, chromosome1, chromosome2):
        for i in range(0 , len(chromosome1) , 3 * self.days):
            prob = random.random()
            if prob <= self.pc:
                chromosome1[i : i + 3 * self.days], chromosome2[i : i + 3 * self.days] = self.onePointCrossOver(chromosome1[i : i + 3 * self.days], chromosome2[i : i + 3 * self.days])
        return chromosome1, chromosome2        

    def mutate(self, chromosome):
        for i in range(len(chromosome)):
            prob = random.random()
            if prob <= self.pm:
                chromosome[i] = 1 - chromosome[i]
        return chromosome

    def calculateFitness(self, chromosome):
        shiftsPlanedDoctor = []
        for i in range(self.days):
            shiftsPlanedDoctor.append([0 , 0 , 0])
        fitnessVal = 0
        for i in range(0, len(chromosome), 3 * self.days):
            doctorShiftsCnt = 0
            checkThreeNight = True
            checkNightAndDay = True
            for j in range(i , i + 3 * self.days , 3):
                for k in range(j , j + 3):
                    if chromosome[k] == 1:
                        shiftsPlanedDoctor[(j - i) // 3][k - j] += 1
                        doctorShiftsCnt += 1
                    if (k != j + 2) and (chromosome[k] == 1) and (j != i) and chromosome[j - 1] == 1:
                        checkNightAndDay = False
                    if (k == j + 2) and (chromosome[k] == 1) and (j - i >= 6) and (chromosome[j - 1] == 1) and (chromosome[j - 4] == 1):
                        checkThreeNight = False
            if doctorShiftsCnt <= self.maxCapacity:
                fitnessVal += 1
            if checkNightAndDay == True:
                fitnessVal += 1
            if checkThreeNight == True:
                fitnessVal += 1
        tmp = 0
        for i in range(self.days):
            for j in range(3):
                if shiftsPlanedDoctor[i][j] >= self.allShifts[i][j][0] and shiftsPlanedDoctor[i][j] <= self.allShifts[i][j][1]:
                    tmp += 1
        fitnessVal += (tmp // 3)
        return fitnessVal        

    def selection(self):
        total = (self.popSize * (self.popSize + 1)) / 2
        backup = self.chromosomes.copy()
        for i in range(6 , self.popSize):
            prob = random.randint(1 , total)
            tmp = self.popSize
            sel_ind = 0
            while tmp != 0:
                if prob <= tmp:
                    self.chromosomes[i] = backup[sel_ind].copy()
                    break
                else:
                    prob -= tmp
                    sel_ind += 1
                    tmp -= 1

    def generateNewPopulation(self):
        self.selection()
        random.shuffle(self.chromosomes)
        for i in range(0 , len(self.chromosomes) , 2):
            self.chromosomes[i], self.chromosomes[i + 1] = self.crossOver(self.chromosomes[i], self.chromosomes[i + 1])
        for i in range(0 , len(self.chromosomes) , 2):
            self.chromosomes[i] = self.mutate(self.chromosomes[i])  
        self.chromosomes.sort(reverse = True, key = self.calculateFitness)   

    def showSchedule(self , chromosome , outputFile):
        out = open(outputFile , 'w')
        shiftsPlanedDoctor = []
        for i in range(self.days):
            shiftsPlanedDoctor.append([[] , [] , []])
        for i in range(0, len(chromosome), 3 * self.days):
            for j in range(i , i + 3 * self.days , 3):
                for k in range(j , j + 3): 
                    if chromosome[k] == 1:
                        shiftsPlanedDoctor[(j - i) // 3][k - j].append(i // (3 * self.days))
        for i in range(self.days):
            for j in range(3):
                if len(shiftsPlanedDoctor[i][j]) == 0:
                    out.write('empty ')
                elif len(shiftsPlanedDoctor[i][j]) == 1:
                    out.write(str(shiftsPlanedDoctor[i][j][0]) + ' ')
                else:
                    out.write(str(shiftsPlanedDoctor[i][j][0]))
                    for k in range(1 , len(shiftsPlanedDoctor[i][j])):
                        out.write(',' + str(shiftsPlanedDoctor[i][j][k]))
                    out.write(' ')
            out.write('\n')
        out.close()

    def schedule(self , outputFile):
        self.chromosomes.sort(reverse = True, key = self.calculateFitness) 
        while (self.calculateFitness(self.chromosomes[0]) != self.goalFitnessVal):
            self.generateNewPopulation()
        self.showSchedule(self.chromosomes[0], outputFile)

<h2 style = "font-size: 25px;">Execution</h2>

In [5]:
import time

fileInfo1 = readInput(testFile1)

start = time.time()

scheduler = JobScheduler(fileInfo1)
scheduler.schedule(outputFile1)

end = time.time()

print("test 1: ", '%.2f'%(end - start), 'sec')

test 1:  3.03 sec


In [6]:
fileInfo2 = readInput(testFile2)

start = time.time()

scheduler = JobScheduler(fileInfo2)
scheduler.schedule(outputFile2)

end = time.time()

print("Test 2: ", '%.2f'%(end - start), 'sec')

Test 2:  21.11 sec
